In [1]:
# Import the dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

In [3]:
# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key

In [4]:
# Import the time library and the datetime module from the datetime library 
import time
from datetime import datetime

In [5]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key


In [6]:
# Create a set of random latitude and longitude combinations

lats = np.random.uniform(low = 90.000, high= 90.000, size =7000)
lngs = np.random.uniform(low = -180.000, high = 180.000, size = 7000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [7]:
# add the latitudes and longitudes to a list

coordinates = list(lat_lngs)

In [8]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

27

In [9]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | narsaq
Processing Record 2 of Set 1 | barrow
Processing Record 3 of Set 1 | belushya guba
City not found. Skipping...
Processing Record 4 of Set 1 | vardo
Processing Record 5 of Set 1 | barentsburg
City not found. Skipping...
Processing Record 6 of Set 1 | tuktoyaktuk
Processing Record 7 of Set 1 | chokurdakh
Processing Record 8 of Set 1 | pevek
Processing Record 9 of Set 1 | cherskiy
Processing Record 10 of Set 1 | khatanga
Processing Record 11 of Set 1 | longyearbyen
Processing Record 12 of Set 1 | ilulissat
Processing Record 13 of Set 1 | qaanaaq
Processing Record 14 of Set 1 | saskylakh
Processing Record 15 of Set 1 | amderma
City not found. Skipping...
Processing Record 16 of Set 1 | illoqqortoormiut
City not found. Skipping...
Processing Record 17 of Set 1 | dikson
Processing Record 18 of Set 1 | mys shmidta
City not found. Skipping...
Processing Record 19 of Set 1 | yellowknife
Processing 

In [10]:
len(city_data)

20

In [11]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Narsaq,60.9167,-46.0500,41.22,76,100,4.41,GL,2022-04-03 20:11:58
1,Barrow,71.2906,-156.7887,-7.58,71,40,16.11,US,2022-04-03 20:11:53
2,Vardo,70.3705,31.1107,26.58,86,75,4.61,NO,2022-04-03 20:07:36
3,Tuktoyaktuk,69.4541,-133.0374,-11.16,95,99,4.05,CA,2022-04-03 20:11:55
4,Chokurdakh,70.6333,147.9167,-21.78,100,81,2.57,RU,2022-04-03 20:07:19
5,Pevek,69.7008,170.3133,-19.73,96,24,2.28,RU,2022-04-03 20:08:40
6,Cherskiy,68.7500,161.3000,-2.67,98,100,3.60,RU,2022-04-03 20:11:59
7,Khatanga,71.9667,102.5000,-29.15,100,99,4.09,RU,2022-04-03 20:11:58
8,Longyearbyen,78.2186,15.6401,8.44,52,20,21.85,SJ,2022-04-03 20:07:26
9,Ilulissat,69.2167,-51.1000,30.22,74,75,9.22,GL,2022-04-03 20:07:05


In [12]:
# rearrange df columns
new_column_order = ["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed"]
city_data_df = city_data_df[new_column_order]
city_data_df

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Narsaq,GL,2022-04-03 20:11:58,60.9167,-46.0500,41.22,76,100,4.41
1,Barrow,US,2022-04-03 20:11:53,71.2906,-156.7887,-7.58,71,40,16.11
2,Vardo,NO,2022-04-03 20:07:36,70.3705,31.1107,26.58,86,75,4.61
3,Tuktoyaktuk,CA,2022-04-03 20:11:55,69.4541,-133.0374,-11.16,95,99,4.05
4,Chokurdakh,RU,2022-04-03 20:07:19,70.6333,147.9167,-21.78,100,81,2.57
5,Pevek,RU,2022-04-03 20:08:40,69.7008,170.3133,-19.73,96,24,2.28
6,Cherskiy,RU,2022-04-03 20:11:59,68.7500,161.3000,-2.67,98,100,3.60
7,Khatanga,RU,2022-04-03 20:11:58,71.9667,102.5000,-29.15,100,99,4.09
8,Longyearbyen,SJ,2022-04-03 20:07:26,78.2186,15.6401,8.44,52,20,21.85
9,Ilulissat,GL,2022-04-03 20:07:05,69.2167,-51.1000,30.22,74,75,9.22


In [14]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")